In [362]:
import pandas as pd
from pyshark import FileCapture
import os

In [376]:
file_path = "/media/HDD/yhk/pcaps/webshell/"

In [389]:
def http_analysis(file_path,file_name):
    """
        解析pcap文件中的全部http流量，并且将request和reponse一一对应
        
    """
    print(file_path+file_name," analysising ...")
    pcaps = FileCapture(file_path+file_name,display_filter="http")

    #=========获取request包相关信息==============#
    request_list = []
    for pkt in pcaps:

        if hasattr(pkt,"http"):
            if hasattr(pkt.http,"request"):
                request = []
                if pkt.http.request_method=="GET":
                    request.extend([pkt.ip.src,pkt.tcp.srcport,pkt.ip.dst,pkt.tcp.dstport,pkt.http.request_method,pkt.http.request_uri,"null"])
                if pkt.http.request_method=="POST":
                    request.extend([pkt.ip.src,pkt.tcp.srcport,pkt.ip.dst,pkt.tcp.dstport,pkt.http.request_method,pkt.http.request_uri,pkt.http.file_data])

                request.extend([pkt.tcp.nxtseq,pkt.http.connection])

                request_list.append(request)

    request_columns = ["srcIp","srcPort","dstIp","dstPort","request_method","requestURL","requestBodyString","nextseq","connection"]
    request_df = pd.DataFrame(request_list,columns=request_columns)

    #==========获取response包相关信息=================#

    response_list = []
    for pkt in pcaps:

        if hasattr(pkt,"http"):
            if hasattr(pkt.http,"response"):
                response= []
                response.extend([pkt.ip.src,pkt.tcp.srcport,pkt.ip.dst,pkt.tcp.dstport])
                response.extend([pkt.http.connection,pkt.tcp.ack,pkt.tcp.flags_push,pkt.http.file_data])
                response_list.append(response)

    response_columns = ["srcIp","srcPort","dstIp","dstPort","connnection","ack","push","responseBodyString"]
    response_df = pd.DataFrame(response_list,columns=response_columns)


    #==========将response转换成可以进行合并的格式=======#
    response_df.columns = ["dstIp","dstPort","srcIp","srcPort","connnection","nextseq","push","responseBodyString"]

    #============response和request进行合并===========#
    df = request_df.merge(response_df,how='left',on=["srcIp","srcPort","dstIp","dstPort","nextseq"])
    df.to_csv("/media/HDD/yhk/pcaps/webshell_analysis_result/"+file_name.split(".")[0]+".csv")
    print(file_path+file_name," completed!")
    return df

In [390]:
for dirpath, dirnames, filenames in os.walk(file_path):
    for i in range(len(filenames)):       
        http_analysis(dirpath,filenames[i])

/media/HDD/yhk/pcaps/webshell/http_test.pcap  analysising ...
/media/HDD/yhk/pcaps/webshell/http_test.pcap  completed!
